In [1]:
import os
import yaml
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dbinsrisk.kics_qis4_scenario import SmithWilson_ALPHA, SmithWilsonYTM, SmithWilson, Cont2Discrete, YTMPrice

In [8]:
# 환경설정
## 환경변수
T = np.arange(1201)
T_OUT = T/12
with open('config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
CURRENCY = config['Currency']
BASE_YYMM = config['BaseYymm']
FILENAME_BASE = config['FileNameBase']
FILENAME_SHOCK = config['FileNameShock']

## 기초정보
with open('settings.json', 'r') as f:
    settings = json.load(f)
ALPHA0 = settings[BASE_YYMM][CURRENCY]['alpha0']
LTFR = settings[BASE_YYMM][CURRENCY]['ltfr']
FREQ = settings[BASE_YYMM][CURRENCY]['freq']
SPREAD = settings[BASE_YYMM][CURRENCY]['spread']
CP = settings[BASE_YYMM][CURRENCY]['cp']
TOL = settings[BASE_YYMM][CURRENCY]['tol']
LLP = settings[BASE_YYMM][CURRENCY]['llp']
TENOR0 = np.array(settings[BASE_YYMM][CURRENCY]['tenor0'], dtype='float')
EOM = datetime.strptime(settings[BASE_YYMM][CURRENCY]['eom'], '%Y-%m-%d')

## 기타설정
os.makedirs('result', exist_ok=True)

In [3]:
# 데이터 불러오기
## 수익률
yield_curve = (
pd.read_excel(f'data/{FILENAME_BASE}')
    .query('base_date == @EOM')
    .query('currency == @CURRENCY')
    .drop(['base_date', 'currency'], axis=1)
    .dropna(axis=1)
)
yield_curve.columns = yield_curve.columns.astype(float)
assert(len(yield_curve) == 1), '데이터 유일성 조건 위배'
ytm = np.array(yield_curve[TENOR0].iloc[0], dtype='float')

## 충격시나리오
shock_cont_all = (
pd.read_excel(f'data/{FILENAME_SHOCK}', dtype={'base_yymm': str})
    .query('base_yymm == @BASE_YYMM')
    .query('currency == @CURRENCY')
    .drop(['base_yymm', 'currency'], axis=1)
    .dropna(axis=1)
    .set_index('scen_no')
)
shock_cont_all.columns = shock_cont_all.columns.astype(float)
shock_cont_all = shock_cont_all[TENOR0[TENOR0<=LLP]]
assert shock_cont_all.index.is_unique, "충격시나리오 유일성 조건 위배"

In [4]:
# Yield2Spot
spot_cont = SmithWilsonYTM(ytm[TENOR0.argmax()], ALPHA0, TENOR0, ytm, FREQ, TENOR0)[TENOR0<=LLP]
tenor = TENOR0[TENOR0<=LLP]

# Liability
shock_cont_all.loc[1] = shock_cont = np.zeros(len(tenor))
shock_cont_all = shock_cont_all.sort_index()
sw_input = np.log(np.exp(spot_cont+shock_cont)+SPREAD)
alpha = SmithWilson_ALPHA(LTFR, tenor, sw_input, CP, TOL)
spot_disc_liab_base = Cont2Discrete(SmithWilson(LTFR, alpha, tenor, sw_input, T_OUT))
forward_disc_liab_base = (1+spot_disc_liab_base[1:])**T[1:]/(1+spot_disc_liab_base[:-1])**T[:-1]-1

# Asset
spot_disc_asset_base = Cont2Discrete(SmithWilsonYTM(ytm[TENOR0.argmax()], ALPHA0, TENOR0, ytm, FREQ, T_OUT))
forward_disc_asset_base = (1+spot_disc_asset_base[1:])**T[1:]/(1+spot_disc_asset_base[:-1])**T[:-1]-1

result_forward_disc_liab, result_forward_disc_asset = [], []
result_spot_disc_liab, result_spot_disc_asset = [], []
for scen_no in tqdm(shock_cont_all.index):
    # LTFR
    lrfr_apply = LTFR+0.0015 if scen_no == 3 else LTFR-0.0015 if scen_no == 4 else LTFR
    
    # Shock
    shock_cont_all.columns = shock_cont_all.columns.astype(float)
    shock_cont = shock_cont_all.loc[scen_no, tenor].values
    sw_input = np.log(np.exp(spot_cont+shock_cont)+SPREAD)
   
    # Liability
    alpha = SmithWilson_ALPHA(lrfr_apply, tenor, sw_input, CP, TOL)
    spot_disc_liab = Cont2Discrete(SmithWilson(lrfr_apply, alpha, tenor, sw_input, T_OUT))
    forward_disc_liab = (1+spot_disc_liab[1:])**T[1:]/(1+spot_disc_liab[:-1])**T[:-1]-1

    # Asset
    spot_disc_asset = spot_disc_asset_base + (spot_disc_liab - spot_disc_liab_base)
    forward_disc_asset = (1+spot_disc_asset[1:])**T[1:]/(1+spot_disc_asset[:-1])**T[:-1]-1

    result_forward_disc_liab.append(forward_disc_liab)
    result_forward_disc_asset.append(forward_disc_asset)
    result_spot_disc_liab.append(spot_disc_liab)
    result_spot_disc_asset.append(spot_disc_asset)

100%|██████████| 6/6 [00:01<00:00,  3.00it/s]


In [5]:
# 데이터 후처리 (1)
## KICS_USER_IR_SCENARIO_01, KICS_USER_IR_SCENARIO_02
mats = [f'M{x:03d}' for x in T[1:]]
kics_user_ir_scenario_liab = pd.DataFrame(np.r_[result_forward_disc_liab])
kics_user_ir_scenario_liab.columns = mats
kics_user_ir_scenario_liab.insert(0, 'SCEN_NO', shock_cont_all.index)
kics_user_ir_scenario_liab.insert(0, 'FSS_SCEN_TYP', '213Z')

kics_user_ir_scenario_asset = pd.DataFrame(np.r_[result_forward_disc_asset])
kics_user_ir_scenario_asset.columns = mats
kics_user_ir_scenario_asset.insert(0, 'SCEN_NO', shock_cont_all.index)
kics_user_ir_scenario_asset.insert(0, 'FSS_SCEN_TYP', '1002')

kics_user_ir_scenario = pd.concat([kics_user_ir_scenario_liab, kics_user_ir_scenario_asset], axis=0).reset_index(drop=True)
kics_user_ir_scenario.insert(0, 'CUR_CD', CURRENCY)
kics_user_ir_scenario.insert(0, 'BSE_DT', EOM.strftime('%Y%m%d'))
kics_user_ir_scenario['LAST_MODIFIED_BY'] = '11700205'
kics_user_ir_scenario['LAST_UPDATE_DATE'] = ''
kics_user_ir_scenario = kics_user_ir_scenario[['BSE_DT', 'FSS_SCEN_TYP', 'SCEN_NO', 'CUR_CD'] + mats + ['LAST_MODIFIED_BY', 'LAST_UPDATE_DATE']]

kics_user_ir_scenario_01 = kics_user_ir_scenario.drop(mats[600:], axis=1)
kics_user_ir_scenario_02 = kics_user_ir_scenario.drop(mats[:600], axis=1)

In [6]:
# 데이터 후처리 (2)
## KICS_USER_FSS_SCENARIO
kics_user_fss_scenario_liab = pd.DataFrame(np.r_[result_spot_disc_liab])
kics_user_fss_scenario_liab.columns = T
kics_user_fss_scenario_liab.insert(0, 'KICS_SCEN_NO', shock_cont_all.index)
kics_user_fss_scenario_liab.insert(0, 'FSS_SCEN_TYP', '3100')
kics_user_fss_scenario_asset = pd.DataFrame(np.r_[result_spot_disc_asset])
kics_user_fss_scenario_asset.columns = T
kics_user_fss_scenario_asset.insert(0, 'KICS_SCEN_NO', shock_cont_all.index)
kics_user_fss_scenario_asset.insert(0, 'FSS_SCEN_TYP', '1100')

kics_user_fss_scenario = pd.concat([kics_user_fss_scenario_liab, kics_user_fss_scenario_asset], axis=0)
kics_user_fss_scenario.insert(0, 'CUR_CD', CURRENCY)
kics_user_fss_scenario.insert(0, 'BSE_YM', BASE_YYMM)
kics_user_fss_scenario = (
kics_user_fss_scenario 
    .melt(id_vars=['BSE_YM', 'FSS_SCEN_TYP', 'CUR_CD', 'KICS_SCEN_NO'], var_name='MAT_TERM', value_name='SPOT_RATE') 
    .query('MAT_TERM != 0')
    .sort_values(by=['BSE_YM', 'FSS_SCEN_TYP', 'CUR_CD', 'KICS_SCEN_NO', 'MAT_TERM'])
    .reset_index(drop=True)
    .assign(LAST_MODIFIED_BY = lambda x: '11700205')
    .assign(LAST_UPDATE_DATE = lambda x: '')
)

In [7]:
# 데이터 내보내기
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/KICS_USER_IR_SCENARIO_{CURRENCY}_{BASE_YYMM}_{now}.xlsx', 'xlsxwriter') as writer:
    kics_user_ir_scenario.to_excel(writer, index=False)
with pd.ExcelWriter(f'result/KICS_USER_IR_SCENARIO_01_{CURRENCY}_{BASE_YYMM}_{now}.xlsx', 'xlsxwriter') as writer:
    kics_user_ir_scenario_01.to_excel(writer, sheet_name='KICS_USER_IR_SCENARIO_01', index=False)
with pd.ExcelWriter(f'result/KICS_USER_IR_SCENARIO_02_{CURRENCY}_{BASE_YYMM}_{now}.xlsx', 'xlsxwriter') as writer:
    kics_user_ir_scenario_02.to_excel(writer, sheet_name='KICS_USER_IR_SCENARIO_02', index=False)
with pd.ExcelWriter(f'result/KICS_USER_FSS_SCENARIO_{CURRENCY}_{BASE_YYMM}_{now}.xlsx', 'xlsxwriter') as writer:
    kics_user_fss_scenario.to_excel(writer, sheet_name='KICS_USER_FSS_SCENARIO', index=False)